In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inference-llama-3-8b/__results__.html
/kaggle/input/inference-llama-3-8b/submission.csv
/kaggle/input/inference-llama-3-8b/__notebook__.ipynb
/kaggle/input/inference-llama-3-8b/__output__.json
/kaggle/input/inference-llama-3-8b/custom.css
/kaggle/input/llama-3/pytorch/8b-chat/1/consolidated.00.pth
/kaggle/input/llama-3/pytorch/8b-chat/1/LICENSE
/kaggle/input/llama-3/pytorch/8b-chat/1/MODEL_CARD.md
/kaggle/input/llama-3/pytorch/8b-chat/1/params.json
/kaggle/input/llama-3/pytorch/8b-chat/1/model.py
/kaggle/input/llama-3/pytorch/8b-chat/1/USE_POLICY.md
/kaggle/input/llama-3/pytorch/8b-chat/1/Llama3_Repo.jpeg
/kaggle/input/llama-3/pytorch/8b-chat/1/example_text_completion.py
/kaggle/input/llama-3/pytorch/8b-chat/1/test_tokenizer.py
/kaggle/input/llama-3/pytorch/8b-chat/1/requirements.txt
/kaggle/input/llama-3/pytorch/8b-chat/1/tokenizer.py
/kaggle/input/llama-3/pytorch/8b-chat/1/eval_details.md
/kaggle/input/llama-3/pytorch/8b-chat/1/generation.py
/kaggle/input/llama-3/pytorc

In [2]:
!pip install streamlit
!pip install pyngrok
!pip install pdfplumber
!pip install pdf2image
!pip install pytesseract
!pip install pillow
!pip install llama-cpp-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 48.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 25.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-

In [3]:
!pip install pyngrok


In [4]:
import os
import re
import json
import pandas as pd
from pypdf import PdfReader
import streamlit as st
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

# Load the LLaMA model and tokenizer from the Kaggle input directory
MODEL_PATH = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(MODEL_PATH)

# Function to extract text from a PDF file
def get_pdf_text(pdf_doc):
    text = ""
    pdf_reader = PdfReader(pdf_doc)
    for page in pdf_reader.pages:
        extracted_text = page.extract_text()
        if extracted_text:
            text += extracted_text + "\n"
    return text

# Function to call the LLaMA model for invoice data extraction
def call_llama_api(pages_data):
    prompt_template = '''Extract the following fields from the invoice data: 
    - Invoice No.
    - Quantity
    - Date
    - Amount
    - Total
    - Email
    - Address
    - Taxable Value
    - SGST Amount
    - CGST Amount
    - IGST Amount
    - SGST Rate
    - CGST Rate
    - IGST Rate
    - Tax Amount
    - Tax Rate
    - Final Amount
    - Invoice Date
    - Place of Supply
    - Place of Origin
    - GSTIN Supplier
    - GSTIN Recipient

    Provide the output in JSON format. Here is the invoice data: {pages}
    '''
    prompt = prompt_template.format(pages=pages_data)

    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    generation_config = GenerationConfig(max_length=500, temperature=0.4)
    
    outputs = model.generate(inputs, generation_config=generation_config)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return output_text

# Function to process multiple PDF files and extract invoice data into a DataFrame
def create_docs(user_pdf_list):
    df = pd.DataFrame({
        'Invoice No.': pd.Series(dtype='str'),
        'Quantity': pd.Series(dtype='str'),
        'Date': pd.Series(dtype='str'),
        'Amount': pd.Series(dtype='float'),
        'Total': pd.Series(dtype='str'),
        'Email': pd.Series(dtype='str'),
        'Address': pd.Series(dtype='str'),
        'Taxable Value': pd.Series(dtype='float'),
        'SGST Amount': pd.Series(dtype='float'),
        'CGST Amount': pd.Series(dtype='float'),
        'IGST Amount': pd.Series(dtype='float'),
        'SGST Rate': pd.Series(dtype='float'),
        'CGST Rate': pd.Series(dtype='float'),
        'IGST Rate': pd.Series(dtype='float'),
        'Tax Amount': pd.Series(dtype='float'),
        'Tax Rate': pd.Series(dtype='float'),
        'Final Amount': pd.Series(dtype='float'),
        'Invoice Date': pd.Series(dtype='str'),
        'Place of Supply': pd.Series(dtype='str'),
        'Place of Origin': pd.Series(dtype='str'),
        'GSTIN Supplier': pd.Series(dtype='str'),
        'GSTIN Recipient': pd.Series(dtype='str'),
    })
    
    for file in user_pdf_list:
        st.write(f"Processing {file.name}...")
        try:
            raw_data = get_pdf_text(file)
            if not raw_data.strip():
                st.warning(f"No text extracted from {file.name}. Skipping.")
                continue
            
            llama_response = call_llama_api(raw_data)
            if not llama_response:
                st.error(f"Failed to extract data from {file.name}.")
                continue

            # Parse the response and find the dictionary structure using regex
            pattern = r'\{(.+)\}'
            match = re.search(pattern, llama_response, re.DOTALL)
            if match:
                extracted_text = match.group(1)
                try:
                    # Safely evaluate the extracted text to a dictionary
                    data_dict = json.loads(f"{{{extracted_text}}}")
                    st.write(f"Extracted Data from {file.name}: {data_dict}")
                except Exception as e:
                    st.error(f"Error parsing extracted data from {file.name}: {e}")
                    continue
            else:
                st.error(f"No valid JSON found in the response for {file.name}.")
                continue

            # Add the extracted data to the DataFrame
            df = pd.concat([df, pd.DataFrame([data_dict])], ignore_index=True)
            st.write("*******DONE******")
        except Exception as e:
            st.error(f"An error occurred while processing {file.name}: {e}")

    # Save the DataFrame to an Excel file
    output_excel_file = "extracted_invoice_data.xlsx"
    df.to_excel(output_excel_file, index=False)
    st.download_button(
        "Download Extracted Data as Excel",
        output_excel_file,
        "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )
    
    return df


# Streamlit app function
def main():
    st.set_page_config(page_title="Invoice Extraction Bot")
    st.title("Invoice Extraction Bot...💁")
    st.subheader("I can help you in extracting invoice data")
    
    # Upload the invoices (PDF files)
    pdf_files = st.file_uploader(
        "Upload invoices here, only PDF files allowed",
        type=["pdf"],
        accept_multiple_files=True
    )
    
    if st.button("Extract Data"):
        if pdf_files:
            with st.spinner('Processing...'):
                df = create_docs(pdf_files)
                
                if not df.empty:
                    st.write("### Extracted Data:")
                    st.dataframe(df)
                    
                    # Allow downloading the extracted data as a CSV file
                    data_as_csv = df.to_csv(index=False).encode('utf-8')
                    st.download_button(
                        "Download data as CSV", 
                        data_as_csv, 
                        "invoice_data.csv", 
                        "text/csv", 
                        key="download-invoice-csv"
                    )
                    st.success("Extraction complete! ✅")
                else:
                    st.warning("No data extracted from the uploaded PDFs.")
        else:
            st.error("Please upload at least one PDF file.")

# Run the app
if __name__ == "__main__":
    main()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


TypeError: not a string

In [ ]:
from transformers import PreTrainedTokenizerFast, LlamaForCausalLM, GenerationConfig

# Load the model and tokenizer from the Kaggle path
MODEL_PATH = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

# Using PreTrainedTokenizerFast for better compatibility with SentencePiece models
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(MODEL_PATH)

# Function to extract invoice data
def call_llama_api(pages_data):
    prompt_template = '''Extract the following fields from the invoice data...'''
    prompt = prompt_template.format(pages=pages_data)

    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    generation_config = GenerationConfig(max_length=500, temperature=0.4)
    
    outputs = model.generate(inputs, generation_config=generation_config)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return output_text


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pip install transformers pdfplumber


In [4]:
from pyngrok import ngrok

# Replace this with your actual ngrok auth token
ngrok.set_auth_token("2nKLK5h8I2R07l44WNljQ8judbX_7Ze6AXjJCxRaCQvXSYPFg")



In [7]:
from pyngrok import ngrok

# Open a tunnel on the desired port
public_url = ngrok.connect(8501, "http")  # Use the port number directly without quotes and specify protocol
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://019f-34-90-36-47.ngrok-free.app" -> "http://localhost:8501"


In [11]:
!pip install llama-cpp-python


In [14]:
import os
import streamlit as st
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import tempfile
import re
import json
from llama_cpp import Llama

# Set page configuration
st.set_page_config(page_title="Invoice Data Extraction", layout="wide")

# Initialize LLaMA model
@st.cache_resource
def load_llama_model():
    model_path = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
    
    # Check if model path exists
    if not os.path.exists(model_path):
        st.error("Model path does not exist or is not accessible.")
        return None
    
    try:
        # Adjust batch size and context to prevent memory issues
        llm = Llama(
            model_path=model_path,
            n_ctx=1024,  # Reduce context size
            n_batch=128,  # Reduce batch size
            use_mlock=False  # Disable memory locking
        )
        return llm
    except Exception as e:
        st.error(f"Error loading LLaMA model: {e}")
        return None


llm = load_llama_model()

# Helper functions
def extract_text_from_pdf(file):
    text = ""
    try:
        with pdfplumber.open(file) as pdf:
            for page in pdf.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + "\n"
    except Exception as e:
        st.error(f"Error reading PDF: {e}")
        return ""
    return text

def extract_text_via_ocr(file):
    text = ""
    with tempfile.TemporaryDirectory() as path:
        try:
            images = convert_from_path(file, output_folder=path)
        except Exception as e:
            st.error(f"Error converting PDF to images: {e}")
            return ""
        for img in images:
            try:
                text += pytesseract.image_to_string(img) + "\n"
            except Exception as e:
                st.error(f"Error with OCR on image: {e}")
                return ""
    return text

def preprocess_text(text):
    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_invoice_data(text):
    prompt = f"""
You are a model designed to extract invoice information. Please strictly extract and return the invoice information in the following JSON format, and nothing else:

JSON Format:
{{
    "Invoice Number": "",
    "Invoice Date": "",
    "Due Date": "",
    "Total Amount": "",
    "Vendor Name": "",
    "Vendor Address": "",
    "Buyer Name": "",
    "Buyer Address": "",
    "GSTIN Number": "",
    "Bank Details": {{
        "Name": "",
        "Account Number": "",
        "IFSC Code": ""
    }},
    "Payment Terms": ""
}}

Extract the information strictly from the invoice text provided below:

\"\"\\""{text}\"\"\\"\"

Ensure that the JSON is valid and strictly adheres to the format. Return no additional explanations.
"""
    if llm:
        try:
            response = llm(
                prompt,
                max_tokens=1024,
                temperature=0.0,
                stop=["\"\"\""]
            )
            return response
        except Exception as e:
            st.error(f"Error during LLaMA model inference: {e}")
            return None
    else:
        st.error("LLaMA model is not loaded.")
        return None

def parse_json(text):
    try:
        # Use regex to find the first JSON-like structure
        json_text_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_text_match:
            json_text = json_text_match.group()
            # Try loading the JSON
            data = json.loads(json_text)
            return data
        else:
            raise ValueError("No valid JSON found in the text.")
    except json.JSONDecodeError as e:
        st.error(f"JSON decode error: {e}")
        return None
    except Exception as e:
        st.error(f"Failed to parse JSON: {e}")
        return None

# Streamlit App
st.title("📄 Invoice Data Extraction")
st.write("Upload your invoice PDFs (Regular, Scanned, or Mixed) to extract relevant information.")

uploaded_file = st.file_uploader("Choose a PDF file", type=["pdf"])

if uploaded_file is not None:
    with st.spinner("Processing..."):
        # Save uploaded file to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(uploaded_file.read())
            tmp_file_path = tmp_file.name

        # Attempt to extract text directly
        text = extract_text_from_pdf(tmp_file_path)

        # If no text found, perform OCR
        if not text.strip():
            text = extract_text_via_ocr(tmp_file_path)

        text = preprocess_text(text)

        if not text.strip():
            st.error("Failed to extract text from the PDF.")
        else:
            st.subheader("Extracted Text")
            st.text_area("Raw Extracted Text", text, height=300)

            # Extract structured data using LLaMA model
            model_response = extract_invoice_data(text)

            if model_response is None:
                st.error("Model inference failed.")
            else:
                # Debugging: Display the raw model response
                st.subheader("Model Response (Debug)")
                st.text_area("Raw Model Response", str(model_response), height=300)

                # Check if response is a dictionary with choices
                if isinstance(model_response, dict) and 'choices' in model_response:
                    if len(model_response['choices']) > 0:
                        extracted_text = model_response['choices'][0]['text'].strip()
                        extracted_data = parse_json(extracted_text)
                    else:
                        st.error("No choices available in the model response.")
                        extracted_data = None
                else:
                    st.error("Unexpected response format from the model.")
                    extracted_data = None

                if extracted_data:
                    st.subheader("Extracted Invoice Data")
                    st.json(extracted_data)

    # Optionally, display the PDF
    st.subheader("Uploaded PDF")
    st.info("Note: For privacy reasons, the PDF is not displayed here.")


2024-10-12 07:21:27.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.852 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.854 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.855 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 07:21:27.860 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [20]:
# Install necessary packages
!pip install pdfplumber pdf2image pytesseract llama-cpp-python pillow

# If not already installed, install ipywidgets for interactive widgets
!pip install ipywidgets

# Import Libraries
import os
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import tempfile
import re
import json
from llama_cpp import Llama
import ipywidgets as widgets
from IPython.display import display, clear_output


In [21]:
# Check if Tesseract is installed
!which tesseract

# If not installed, install it
# Uncomment the following line if Tesseract is not found
# !apt-get install -y tesseract-ocr


/usr/bin/tesseract


In [29]:
# Initialize LLaMA model
def load_llama_model():
    model_path = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"  # Updated model path
    llm = Llama(
        model_path=model_path,
        n_ctx=2048,
        n_batch=512,
        use_mlock=True  # Adjust based on your environment's capabilities
    )
    return llm

# Load the model
llm = load_llama_model()


gguf_init_from_file: invalid magic characters '�[  '
llama_model_load: error loading model: llama_model_loader: failed to load model from /kaggle/input/llama-3/transformers/8b-chat-hf/1

llama_load_model_from_file: failed to load model


ValueError: Failed to load model from file: /kaggle/input/llama-3/transformers/8b-chat-hf/1

In [30]:
def extract_text_from_pdf(file_path):
    """
    Extracts text from a PDF file using pdfplumber.
    """
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""
    return text

def extract_text_via_ocr(file_path):
    """
    Extracts text from a PDF file using OCR (pytesseract) after converting PDF pages to images.
    """
    text = ""
    with tempfile.TemporaryDirectory() as path:
        try:
            images = convert_from_path(file_path, output_folder=path)
        except Exception as e:
            print(f"Error converting PDF to images: {e}")
            return ""
        for img in images:
            try:
                text += pytesseract.image_to_string(img) + "\n"
            except Exception as e:
                print(f"Error with OCR on image: {e}")
                return ""
    return text

def preprocess_text(text):
    """
    Cleans the extracted text by removing excessive whitespace.
    """
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_invoice_data(text, llm_model):
    """
    Uses the LLaMA model to extract structured invoice data from text.
    """
    prompt = f"""
You are a model designed to extract invoice information. Please strictly extract and return the invoice information in the following JSON format, and nothing else:

JSON Format:
{{
    "Invoice Number": "",
    "Invoice Date": "",
    "Due Date": "",
    "Total Amount": "",
    "Vendor Name": "",
    "Vendor Address": "",
    "Buyer Name": "",
    "Buyer Address": "",
    "GSTIN Number": "",
    "Bank Details": {{
        "Name": "",
        "Account Number": "",
        "IFSC Code": ""
    }},
    "Payment Terms": ""
}}

Extract the information strictly from the invoice text provided below:

\"\"\\""" 
{text} 
\"\"\\""" 

Ensure that the JSON is valid and strictly adheres to the format. Return no additional explanations.
"""
    response = llm_model(
        prompt,
        max_tokens=1024,
        temperature=0.0,
        stop=["\"\"\""]
    )
    return response

def parse_json(text):
    """
    Parses JSON text extracted from the model's response.
    """
    try:
        # Use regex to find the first JSON-like structure
        json_text_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_text_match:
            json_text = json_text_match.group()
            # Try loading the JSON
            data = json.loads(json_text)
            return data
        else:
            raise ValueError("No valid JSON found in the text.")
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None
    except Exception as e:
        print(f"Failed to parse JSON: {e}")
        return None


SyntaxError: unexpected character after line continuation character (3752034076.py, line 73)

In [31]:
import os

model_dir = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
for root, dirs, files in os.walk(model_dir):
    print(root)
    for file in files:
        print("  -", file)


/kaggle/input/llama-3/transformers/8b-chat-hf/1
  - model.safetensors.index.json
  - model-00003-of-00004.safetensors
  - config.json
  - LICENSE
  - model-00001-of-00004.safetensors
  - model.py
  - USE_POLICY.md
  - tokenizer.json
  - tokenizer_config.json
  - example_text_completion.py
  - test_tokenizer.py
  - requirements.txt
  - tokenizer.py
  - model-00004-of-00004.safetensors
  - eval_details.md
  - special_tokens_map.json
  - generation.py
  - model-00002-of-00004.safetensors
  - __init__.py
  - example_chat_completion.py
  - setup.py
  - generation_config.json


In [ ]:
# Clone the repository
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp

# Build the conversion tool
!make


In [36]:
# Example command (adjust based on the actual conversion tool's instructions)
./convert-hf-to-gguf --input /kaggle/input/llama-3/transformers/8b-chat-hf/1 --output /kaggle/input/llama-3/transformers/8b-chat-hf.gguf


SyntaxError: invalid decimal literal (1341952149.py, line 2)

In [37]:
!pip install transformers
!pip install accelerate
!pip install sentencepiece


In [38]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name_or_path = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", torch_dtype=torch.float16)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [40]:
import torch

def extract_invoice_data(text):
    prompt = f"""
You are a model designed to extract invoice information. Please strictly extract and return the invoice information in the following JSON format, and nothing else:

JSON Format:
{{
    "Invoice Number": "",
    "Invoice Date": "",
    "Due Date": "",
    "Total Amount": "",
    "Vendor Name": "",
    "Vendor Address": "",
    "Buyer Name": "",
    "Buyer Address": "",
    "GSTIN Number": "",
    "Bank Details": {{
        "Name": "",
        "Account Number": "",
        "IFSC Code": ""
    }},
    "Payment Terms": ""
}}

Extract the information strictly from the invoice text provided below:

\"\"\" 
{text} 
\"\"\"

Ensure that the JSON is valid and strictly adheres to the format. Return no additional explanations.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=2048, temperature=0.0, stop_token=None)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [24]:
# Install necessary packages
!pip install pdfplumber pdf2image pytesseract transformers accelerate sentencepiece ipywidgets

# Import Libraries
import os
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import tempfile
import re
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
!which tesseract


/usr/bin/tesseract


In [25]:
# Check if Tesseract is installed
tesseract_path = !which tesseract
if tesseract_path:
    print(f"Tesseract is installed at: {tesseract_path[0]}")
else:
    print("Tesseract is not installed. Attempting to install...")
    # Uncomment the following line if Tesseract is not installed and if Kaggle permits
    # !apt-get install -y tesseract-ocr
    print("Please install Tesseract OCR manually or ensure it's available in the environment.")


Tesseract is installed at: /usr/bin/tesseract


In [26]:
# Initialize Hugging Face model
def load_hf_model():
    # Choose a smaller model to fit within GPU memory limits
    model_path = "facebook/opt-6.7b"  # You can change this to another smaller model if needed
    
    try:
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        print("Tokenizer loaded successfully.")
        
        print("Loading model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.float16,
            # load_in_8bit=True  # Uncomment if using 8-bit precision and supported
        )
        print("Model loaded successfully.")
        return tokenizer, model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Load the model
tokenizer, model = load_hf_model()

# Verify model loading
if model is not None and tokenizer is not None:
    print("Model and tokenizer are ready for use.")
else:
    print("Model and/or tokenizer failed to load. Please check the model path and format.")


Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model loaded successfully.
Model and tokenizer are ready for use.


In [14]:
# Initialize Hugging Face model
def load_hf_model():
    model_path = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"  # Update as necessary
    try:
        print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        print("Tokenizer loaded successfully.")
        
        print("Loading model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map="auto",
            torch_dtype=torch.float16,
            # load_in_8bit=True  # Uncomment if using 8-bit precision and supported
        )
        print("Model loaded successfully.")
        return tokenizer, model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Load the model
tokenizer, model = load_hf_model()

# Verify model loading
if model is not None and tokenizer is not None:
    print("Model and tokenizer are ready for use.")
else:
    print("Model and/or tokenizer failed to load. Please check the model path and format.")


Loading tokenizer...
Tokenizer loaded successfully.
Loading model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully.
Model and tokenizer are ready for use.


In [34]:
def extract_text_from_pdf(file_path):
    """
    Extracts selectable text from a PDF file using pdfplumber.
    """
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                extracted = page.extract_text()
                if extracted:
                    text += extracted + "\n"
                else:
                    print(f"No selectable text found on page {page_number}.")
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""
    return text

def extract_text_via_ocr(file_path):
    """
    Extracts text from a PDF file using OCR (pytesseract) after converting PDF pages to images.
    """
    text = ""
    with tempfile.TemporaryDirectory() as path:
        try:
            print("Converting PDF pages to images for OCR...")
            images = convert_from_path(file_path, output_folder=path, fmt='png')
            print(f"Converted PDF to {len(images)} image(s).")
        except Exception as e:
            print(f"Error converting PDF to images: {e}")
            return ""
        for idx, img in enumerate(images, start=1):
            try:
                print(f"Performing OCR on page {idx}...")
                ocr_text = pytesseract.image_to_string(img)
                if ocr_text.strip():
                    text += ocr_text + "\n"
                else:
                    print(f"No text found via OCR on page {idx}.")
            except Exception as e:
                print(f"Error with OCR on image page {idx}: {e}")
                return ""
    return text

def preprocess_text(text):
    """
    Cleans the extracted text by removing excessive whitespace and unwanted characters.
    """
    # Remove multiple spaces, newlines, and tabs
    text = re.sub(r'\s+', ' ', text)
    # Optionally, remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.strip()

def extract_invoice_data(text, tokenizer, model):
    """
    Uses the Hugging Face model to extract structured invoice data from text.
    """
    prompt = f"""
You are an AI assistant specialized in extracting invoice information. Please strictly extract and return the invoice information in the following JSON format, and nothing else:

JSON Format:
{{
    "Invoice Number": "",
    "Invoice Date": "",
    "Due Date": "",
    "Total Amount": "",
    "Vendor Name": "",
    "Vendor Address": "",
    "Buyer Name": "",
    "Buyer Address": "",
    "GSTIN Number": "",
    "Bank Details": {{
        "Name": "",
        "Account Number": "",
        "IFSC Code": ""
    }},
    "Payment Terms": ""
}}

Extract the information strictly from the invoice text provided below:

\"\"\"
{text}
\"\"\"

Ensure that the JSON is valid and strictly adheres to the format. Do not include any explanations or additional text.
"""

    try:
        print("Tokenizing input...")
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        print("Generating model response...")
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=2048,
                temperature=0.0,
                do_sample=False,
                eos_token_id=tokenizer.eos_token_id
            )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print("Model response generated.")
        return response
    except Exception as e:
        print(f"Error during model inference: {e}")
        return ""

def parse_json(text):
    """
    Parses JSON text extracted from the model's response.
    """
    try:
        # Use regex to find the first JSON-like structure
        json_text_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_text_match:
            json_text = json_text_match.group()
            # Try loading the JSON
            data = json.loads(json_text)
            return data
        else:
            raise ValueError("No valid JSON found in the text.")
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None
    except Exception as e:
        print(f"Failed to parse JSON: {e}")
        return None


In [35]:
# Create a file uploader widget
uploader = widgets.FileUpload(
    accept='.pdf',  # Accept only PDF files
    multiple=False,  # Allow single file upload
    description='Upload PDF',
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    style={'description_width': 'initial'},
)

# Create a button to trigger processing
process_button = widgets.Button(
    description="Process Uploaded PDF",
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to process the uploaded PDF file',
    icon='check'  # (FontAwesome names without the `fa-` prefix)
)

# Display the uploader and the button
display(uploader, process_button)


FileUpload(value={}, accept='.pdf', button_style='info', description='Upload PDF')

Button(button_style='success', description='Process Uploaded PDF', icon='check', style=ButtonStyle(), tooltip=…

In [32]:
def process_uploaded_file(uploader_widget, tokenizer, model):
    """
    Processes the uploaded PDF file: extracts text, performs OCR if needed, and extracts invoice data.
    """
    if not uploader_widget.value:
        print("No file uploaded yet. Please upload a PDF file to proceed.")
        return
    
    # Retrieve the uploaded file
    uploaded_filename = list(uploader_widget.value.keys())[0]
    uploaded_file = uploader_widget.value[uploaded_filename]
    file_content = uploaded_file['content']
    
    # Save the uploaded file to a temporary file
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(file_content)
            tmp_file_path = tmp_file.name
        print(f"\nSaved uploaded file as temporary file: {tmp_file_path}")
    except Exception as e:
        print(f"Error saving uploaded file: {e}")
        return
    
    print(f"\nProcessing file: {uploaded_filename}")
    
    # Attempt to extract text directly
    print("\n--- Step 1: Extracting Text Using pdfplumber ---")
    text = extract_text_from_pdf(tmp_file_path)
    
    # If no text found, perform OCR
    if not text.strip():
        print("\n--- Step 2: No Text Found via pdfplumber. Performing OCR ---")
        text = extract_text_via_ocr(tmp_file_path)
    
    # Preprocess the text
    print("\n--- Step 3: Preprocessing Extracted Text ---")
    text = preprocess_text(text)
    
    if not text.strip():
        print("Failed to extract text from the PDF after both extraction methods.")
        return
    else:
        print("\n--- Extracted Text (First 1000 characters) ---\n")
        print(text[:1000] + ('...' if len(text) > 1000 else ''))
    
    # Extract structured data using Hugging Face model
    print("\n--- Step 4: Extracting Structured Invoice Data Using Model ---\n")
    model_response = extract_invoice_data(text, tokenizer, model)
    
    if not model_response:
        print("Model inference failed or returned an empty response.")
        return
    else:
        # Debugging: Display the raw model response
        print("\n--- Model Response (Debug) ---\n")
        print(model_response)
    
        # Parse the JSON from the response
        print("\n--- Step 5: Parsing JSON from Model Response ---\n")
        extracted_data = parse_json(model_response)
    
        if extracted_data:
            print("\n--- Extracted Invoice Data ---\n")
            print(json.dumps(extracted_data, indent=4))
        else:
            print("Failed to parse extracted data from the model response.")


In [31]:
# Define the button's click event handler
def on_process_button_clicked(b):
    # Optionally, clear previous outputs if desired
    # clear_output(wait=True)
    print("\nButton clicked. Starting processing...\n")
    process_uploaded_file(uploader, tokenizer, model)

# Link the button to the event handler
process_button.on_click(on_process_button_clicked)


In [5]:
# Install necessary packages
!pip install pdfplumber pdf2image pytesseract pillow ipywidgets llama-cpp-python

# Install Tesseract OCR
!apt-get install -y tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.


In [6]:
import os
import pdfplumber
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import tempfile
import re
import json
from llama_cpp import Llama
import ipywidgets as widgets
from IPython.display import display, clear_output


In [13]:
!pip install transformers


In [15]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_llama_model():
    model_path = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

    # Check if the model directory exists
    if not os.path.exists(model_path):
        raise ValueError(f"Model path does not exist: {model_path}")

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map='auto')

    return model, tokenizer

# Load the model and tokenizer
llm, tokenizer = load_llama_model()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
def extract_text_from_pdf(file_path):
    text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + "\n"
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""
    return text

def extract_text_via_ocr(file_path):
    text = ""
    with tempfile.TemporaryDirectory() as path:
        try:
            images = convert_from_path(file_path, output_folder=path)
        except Exception as e:
            print(f"Error converting PDF to images: {e}")
            return ""
        for img in images:
            try:
                text += pytesseract.image_to_string(img) + "\n"
            except Exception as e:
                print(f"Error with OCR on image: {e}")
                return ""
    return text

def preprocess_text(text):
    # Remove multiple spaces and newlines
    text = re.sub(r'\s+', ' ', text)
    return text

def extract_invoice_data(text):
    prompt = f"""
You are a model designed to extract invoice information. Please strictly extract and return the invoice information in the following JSON format, and nothing else:

JSON Format:
{{
    "Invoice Number": "",
    "Invoice Date": "",
    "Due Date": "",
    "Total Amount": "",
    "Vendor Name": "",
    "Vendor Address": "",
    "Buyer Name": "",
    "Buyer Address": "",
    "GSTIN Number": "",
    "Bank Details": {{
        "Name": "",
        "Account Number": "",
        "IFSC Code": ""
    }},
    "Payment Terms": ""
}}

Extract the information strictly from the invoice text provided below:

\"\"\" 
{text} 
\"\"\"

Ensure that the JSON is valid and strictly adheres to the format. Return no additional explanations.
"""
    response = llm(
        prompt,
        max_tokens=1024,
        #max_tokens=1024,
        temperature=0.0,
        stop=["\"\"\""]
    )
    return response

def parse_json(text):
    try:
        # Use regex to find the first JSON-like structure
        json_text_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_text_match:
            json_text = json_text_match.group()
            # Try loading the JSON
            data = json.loads(json_text)
            return data
        else:
            raise ValueError("No valid JSON found in the text.")
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None
    except Exception as e:
        print(f"Failed to parse JSON: {e}")
        return None


In [35]:
# Create a file uploader widget
uploader = widgets.FileUpload(
    accept='.pdf',  # Accept PDF files only
    multiple=False  # Single file upload
)

# Display the uploader
display(uploader)


FileUpload(value={}, accept='.pdf', description='Upload')

In [36]:
def process_pdf(change):
    # Check if a file is uploaded
    if uploader.value:
        # Clear previous outputs
        clear_output(wait=True)
        display(uploader)
        
        # Get the uploaded file
        uploaded_file = next(iter(uploader.value.values()))
        file_content = uploaded_file['content']
        file_name = uploaded_file['metadata']['name']
        
        # Save the uploaded file to a temporary file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
            tmp_file.write(file_content)
            tmp_file_path = tmp_file.name
        
        print("📄 Processing the uploaded PDF...")
        
        # Attempt to extract text directly
        text = extract_text_from_pdf(tmp_file_path)
        
        # If no text found, perform OCR
        if not text.strip():
            print("🔍 No text found using pdfplumber. Attempting OCR...")
            text = extract_text_via_ocr(tmp_file_path)
        
        text = preprocess_text(text)
        
        if not text.strip():
            print("❌ Failed to extract text from the PDF.")
            return
        else:
            print("📝 Extracted Text:")
            print("-" * 50)
            print(text)
            print("-" * 50)
        
        # Extract structured data using LLaMA model
        print("🤖 Extracting structured invoice data using LLaMA model...")
        model_response = extract_invoice_data(text)
        
        if model_response is None:
            print("❌ Model inference failed.")
            return
        else:
            # Debugging: Display the raw model response
            print("📤 Raw Model Response:")
            print("-" * 50)
            print(model_response)
            print("-" * 50)
        
            # Check if response is a dictionary with choices
            if isinstance(model_response, dict) and 'choices' in model_response:
                if len(model_response['choices']) > 0:
                    extracted_text = model_response['choices'][0]['text'].strip()
                    extracted_data = parse_json(extracted_text)
                else:
                    print("❌ No choices available in the model response.")
                    extracted_data = None
            else:
                print("❌ Unexpected response format from the model.")
                extracted_data = None
        
            if extracted_data:
                print("✅ Extracted Invoice Data:")
                print(json.dumps(extracted_data, indent=4))
            else:
                print("❌ Failed to extract structured data from the model response.")


In [37]:
# Link the processing function to the uploader's 'value' attribute
uploader.observe(process_pdf, names='value')
